In [31]:
import numpy as np
import pandas as pd
import sklearn.linear_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numerapi
%matplotlib inline
import talib as ta

In [57]:
import pandas_datareader.data as web

tsd=web.DataReader('6758.JP', 'stooq')
tsd = tsd.sort_values('Date')
prices = np.array(tsd["Close"])

# 5 日単純移動平均を求める
sma5 = ta.SMA(prices, timeperiod=5)
tsd["sma5"] =sma5 
#print(sma5)
# RSI (14 日) を求める
rsi14 = ta.RSI(prices, timeperiod=14)
tsd["rsi14"] =rsi14
# MACD (先行 12 日移動平均、遅行 26 日移動平均、 9 日シグナル線) を求める
macd, macdsignal, macdhist = ta.MACD(prices,fastperiod=6, slowperiod=13, signalperiod=4)
tsd["macd"] =macd 
tsd["macdsignal"] =macdsignal 
tsd["macdhist"] =macdhist

tsd

,Open,High,Low,Close,Volume,sma5,rsi14,macd,macdsignal,macdhist
Date,,,,,,,,,,
2015-06-17,3667.62,3680.76,3569.91,3596.64,7190025,NaN,NaN,NaN,NaN,NaN
2015-06-18,3593.23,3620.95,3564.08,3577.68,4557727,NaN,NaN,NaN,NaN,NaN
2015-06-19,3604.89,3631.17,3556.28,3556.28,5123408,NaN,NaN,NaN,NaN,NaN
2015-06-22,3570.86,3679.79,3566.99,3629.20,7418519,NaN,NaN,NaN,NaN,NaN
2015-06-23,3674.43,3721.60,3653.04,3721.60,7611535,3616.28,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-06-08,7377.00,7407.00,7330.00,7396.00,5597200,7245.40,69.959333,141.092673,111.518785,29.573888
2020-06-09,7410.00,7437.00,7332.00,7402.00,5017600,7299.20,70.117867,148.402800,126.272391,22.130409
2020-06-10,7372.00,7524.00,7340.00,7496.00,6131100,7368.20,72.560988,160.249702,139.863315,20.386387


In [58]:
from sklearn import preprocessing
import pandas as pd

tsd['Difference'] = tsd['Close'] - tsd['Open']

x = tsd.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df

#df[10].values 
tsd['target'] = df[10].values 
tsd=tsd.drop('Difference', axis=1)
tsd['target'] =tsd.target.shift(-1)
tsd=tsd.dropna(how='any')
tsd
#df.describe()

,Open,High,Low,Close,Volume,sma5,rsi14,macd,macdsignal,macdhist,target
Date,,,,,,,,,,,
2015-07-08,3350.11,3390.47,3230.50,3230.50,8783306,3347.980,30.877538,-108.929904,-94.953017,-13.976886,0.715755
2015-07-09,3144.94,3311.21,3139.09,3304.90,11769657,3325.418,36.457021,-102.547706,-97.990893,-4.556813,0.478010
2015-07-10,3311.21,3382.21,3299.06,3325.81,13002513,3309.180,37.972415,-91.467474,-95.381525,3.914051,0.496295
2015-07-13,3403.60,3493.55,3400.19,3429.38,17291332,3329.408,44.972486,-66.154578,-83.690747,17.536168,0.336403
2015-07-14,3460.98,3479.93,3337.47,3389.00,25647473,3335.918,42.937903,-53.725272,-71.704557,17.979285,0.378551
...,...,...,...,...,...,...,...,...,...,...,...
2020-06-05,7150.00,7280.00,7146.00,7280.00,5385800,7160.800,66.796960,121.146622,91.802859,29.343762,0.485206
2020-06-08,7377.00,7407.00,7330.00,7396.00,5597200,7245.400,69.959333,141.092673,111.518785,29.573888,0.441046
2020-06-09,7410.00,7437.00,7332.00,7402.00,5017600,7299.200,70.117867,148.402800,126.272391,22.130409,0.656941


In [59]:
training_data = tsd
feature_names = [f for f in training_data.columns if not "target" in f]
feature_names

['Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'sma5',
 'rsi14',
 'macd',
 'macdsignal',
 'macdhist']

In [60]:
# ライブラリ
from sklearn.linear_model import Ridge

# create a model and fit the training data (~30 sec to run)
model = Ridge()

# 説明変数
X= training_data[feature_names]
# 目的変数
y = training_data["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# モデルのあてはめ
model.fit(X_train,y_train)
#print(model.coef_)
#print(model.intercept_)
print("train:",model.__class__.__name__ ,model.score(X_train,y_train))

#　各モデル個別の予測精度
train_pred = model.predict(X_train)
rank_pred1 = y_train.rank(pct=True, method="first")
a = np.corrcoef(train_pred, rank_pred1)[0,1]
print (a)

test_pred = model.predict(X_test)
rank_pred2 = y_test.rank(pct=True, method="first")
b = np.corrcoef(test_pred, rank_pred2)[0,1]
print (b)

train: Ridge 0.047069170979472164
0.14054220547057436
-0.1333507972597674


/Users/nakamuratatsuya/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=7.10889e-17): result may not be accurate.
  overwrite_a=True).T


In [62]:
training_data["prediction"] = model.predict(training_data[feature_names])
ranked_prediction = training_data["prediction"].rank(pct=True, method="first")
correlation = np.corrcoef(training_data.target, ranked_prediction)[0, 1]

#print (ranked_prediction)
print (correlation)

#training_data["prediction"]

0.08582559696513771


In [64]:
tsd

,Open,High,Low,Close,Volume,sma5,rsi14,macd,macdsignal,macdhist,target,prediction
Date,,,,,,,,,,,,
2015-07-08,3350.11,3390.47,3230.50,3230.50,8783306,3347.980,30.877538,-108.929904,-94.953017,-13.976886,0.715755,0.486742
2015-07-09,3144.94,3311.21,3139.09,3304.90,11769657,3325.418,36.457021,-102.547706,-97.990893,-4.556813,0.478010,0.445835
2015-07-10,3311.21,3382.21,3299.06,3325.81,13002513,3309.180,37.972415,-91.467474,-95.381525,3.914051,0.496295,0.479771
2015-07-13,3403.60,3493.55,3400.19,3429.38,17291332,3329.408,44.972486,-66.154578,-83.690747,17.536168,0.336403,0.485582
2015-07-14,3460.98,3479.93,3337.47,3389.00,25647473,3335.918,42.937903,-53.725272,-71.704557,17.979285,0.378551,0.519654
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-05,7150.00,7280.00,7146.00,7280.00,5385800,7160.800,66.796960,121.146622,91.802859,29.343762,0.485206,0.412085
2020-06-08,7377.00,7407.00,7330.00,7396.00,5597200,7245.400,69.959333,141.092673,111.518785,29.573888,0.441046,0.431907
2020-06-09,7410.00,7437.00,7332.00,7402.00,5017600,7299.200,70.117867,148.402800,126.272391,22.130409,0.656941,0.433212


In [63]:
tsd.sort_values('prediction', ascending=False)

,Open,High,Low,Close,Volume,sma5,rsi14,macd,macdsignal,macdhist,target,prediction
Date,,,,,,,,,,,,
2020-03-16,5778.32,5956.65,5579.07,5611.94,17369528,5955.654,22.642214,-368.356285,-327.698294,-40.657991,0.879426,0.630374
2020-03-12,5977.57,6090.15,5797.25,5860.01,14670639,6249.352,25.340310,-317.778054,-272.924039,-44.854015,1.000000,0.612738
2020-03-09,6399.99,6422.90,6112.07,6190.77,12957837,6667.582,25.289538,-242.849256,-208.796093,-34.053163,0.951113,0.602112
2020-03-11,6274.46,6402.98,6099.12,6122.03,12254709,6457.970,28.497267,-273.376620,-243.021362,-30.355258,0.261854,0.594669
2020-02-28,6674.96,6743.70,6547.43,6586.29,11680462,7051.740,22.915394,-233.754638,-172.194800,-61.559839,0.900607,0.583857
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-05,6157.65,6379.12,6147.71,6345.35,9872248,6101.030,70.791506,100.316392,64.554161,35.762231,0.468753,0.401031
2020-04-15,6630.00,6799.00,6581.00,6767.00,6668800,6654.200,56.213194,87.379521,84.354529,3.024992,0.452495,0.395458
2020-04-17,6850.00,6985.00,6791.00,6969.00,9271300,6711.800,60.378142,115.905728,96.733349,19.172379,0.308565,0.391731


In [ ]:
# 利益計算
def cal_profit(y, y_pred):
    # 予測
    y_pred = np.where(y_pred < 10, 0, 1)
    csv_df = pd.DataFrame()
    sum = 0
    for i in range(len(y_pred)):
        if(y_pred[i]):
            csv_df = csv_df.append([y[i]], ignore_index=True)
            sum += y[i]
        else:
            csv_df = csv_df.append([0], ignore_index=True)
    print("total profit = " + str(sum))
    csv_df.to_csv("profit.csv", index = False)

In [ ]:
tsd = web.DataReader("^N225","yahoo","2010/1/1").dropna()#jpy
tsd.loc[:,'Adj Close'].plot()
tsd.to_csv("tsd.csv")

In [ ]:
l = [6758, 6501, 6752]    # ソニー、日立製作所、パナソニック

web.DataReader([str(c) + '.JP' for c in l], 'stooq')

In [ ]:
import pandas_datareader.data as web
#マザーズ
web.DataReader('2516.JP', 'stooq')

In [ ]:
#マザーズ
web.DataReader('2516.JP', 'stooq')